# Calibration Diagnostic

In [ ]:
obs_config = dict(
    wterms = 7,
    cterms = 7,
    f_low = 40,
    f_high = None,
    run_num = None,
    repeat_num = None,
    resistance_f = 50.009,
    load_kwargs = None,
    s11_kwargs = None,
    load_spectra = None,
    load_s11s = None,
    compile_from_def = True,
    include_previous = False,
)
cmp_config = dict(
    wterms = 7,
    cterms = 7,
    run_num = None,
    repeat_num = None,
    resistance_f = 50.009,
    load_kwargs = None,
    s11_kwargs = None,
    load_spectra = None,
    load_s11s = None,
    compile_from_def = True,
    include_previous = False,
)

In [ ]:
# These are the defaults for the run. They are overwritten on command-line.
observation = '/data5/edges/data/CalibrationObservations/Receiver01/Receiver01_25C_2019_10_01_040_to_200MHz'
cmp_observation = '/data5/edges/data/CalibrationObservations/Receiver01/Receiver01_25C_2019_11_26_040_to_200MHz'

obs_config_ = {}  # Need to do it this way, otherwise they get fully over-ridden
cmp_config_ = {}

In [ ]:
obs_config.update(obs_config_)
cmp_config.update(cmp_config_)

In [ ]:
cmp_config['f_low']=obs_config['f_low']
cmp_config['f_high']=obs_config['f_high']

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from datetime import datetime
import edges_io as eio
import edges_cal as ec
from pathlib import Path
import yaml
import warnings
from matplotlib.lines import Line2D
from beautifultable import BeautifulTable

This document compares two observations:

In [ ]:
obspath = Path(observation).absolute()
cmppath = Path(cmp_observation).absolute()
print("Primary Location   : ", obspath)
print("Comparison Location: ", cmppath)
print()

print("Date Run: ", datetime.now())
print("Version of edges-io:", eio.__version__)
print("Version of edges-cal:", ec.__version__)

In [ ]:
calobs = ec.CalibrationObservation(obspath, **obs_config)
calobs_cmp = ec.CalibrationObservation(cmppath, **cmp_config)

In [ ]:
simnames = calobs.io.simulator_names.intersection(calobs_cmp.io.simulator_names)
antsims = {name: calobs.new_load(name) for name in simnames}
antsims_cmp = {name: calobs_cmp.new_load(name) for name in simnames}

## Observation Metadata

The following metadata fully defines this calibration for this version of `edges-cal`:

In [ ]:
def print_summary(calobs, cfg):
    if cfg['compile_from_def'] and calobs.path != obspath:
        if calobs.io.definition:
            print(f"> Compiled from definition file. Definition:\n{yaml.dump(calobs.io.definition)}")
        else:
            print("> Compiled from empty definition file.")
    else:
        print("> Not compiled from definition file.")
    print()

    table = BeautifulTable()
    table.rows.append(['wterms', calobs.wterms, 'cterms', calobs.cterms])
    table.rows.append(['f_min', f"{calobs.freq.min:.2f} MHz", 'f_max', f"{calobs.freq.max:.2f} MHz"])
    table.rows.append(['LNA Res.', f"{calobs.lna.resistance} Ohms", "Includes Previous?", str(cfg['include_previous'])])
    print(table)
    print()

    table = BeautifulTable()
    table.columns.header = [
        'S11 Run', 
        'Spec/Res. Run', 
        'S11 Rep.', 
        'Ignored %', 
        'Male Res. (Ohms)'
    ]
    sources = []
    for name, source in calobs._loads.items():
        sources.append(name)
        table.rows.append([
            source.reflections.run_num,
            source.spectrum.run_num,
            calobs.io.s11.repeat_num[name],
            source.spectrum.ignore_times_percent,
            source.reflections.resistance
        ])
    table.rows.header = sources
    print(table)

In [ ]:
print("Primary Observation:", obspath.name)
print("=====================" + '='*len(obspath.name))
print_summary(calobs, obs_config)

In [ ]:
print("Comparison Observation:", cmppath.name)
print("========================" + '='*len(cmppath.name))
print_summary(calobs_cmp, cmp_config)

In [ ]:
print("Antenna Simulators available in both observations: ", ', '.join(sorted(simnames)))

## Raw Data

In [ ]:
fig, ax = plt.subplots(
    3,4,figsize=(16,8), 
    #gridspec_kw={"hspace":0.05, 'wspace':0.05}
)

for i, (name, load) in enumerate(calobs._loads.items()):
    ax[0,i].plot(load.spectrum.thermistor_temp, label='Primary')
    ax[0,i].plot(calobs_cmp._loads[name].spectrum.thermistor_temp, label='Comparison')
    
    ax[0,i].set_title(name.capitalize(), fontsize=15)
    ax[0,i].set_xlabel("Time")
    
    load.spectrum.plot(fig=fig, ax=ax[1,i],ylabel=False)
    calobs_cmp._loads[name].spectrum.plot(fig=fig, ax=ax[1,i],ylabel=False)
    
    calobs.plot_calibrated_temp(load, ax=ax[2,i],ylabel=False)
    calobs.plot_calibrated_temp(calobs_cmp._loads[name], ax=ax[2,i],ylabel=False)
    
ax[0,0].set_ylabel("Thermistor Temp (K)")
ax[1,0].set_ylabel("Uncalibrated Temp (K)")
ax[2,0].set_ylabel("Calibrated Temp (K)")
ax[0,1].legend()
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(
    3, len(antsims),figsize=(16,8), 
    #gridspec_kw={"hspace":0.05, 'wspace':0.05}
)

for i, (name, load) in enumerate(antsims.items()):
    ax[0,i].plot(load.spectrum.thermistor_temp, label='Primary')
    ax[0,i].plot(antsims_cmp[name].spectrum.thermistor_temp, label='Comparison')
    
    ax[0,i].set_title(name.capitalize(), fontsize=15)
    ax[0,i].set_xlabel("Time")
    
    load.spectrum.plot(fig=fig, ax=ax[1,i],ylabel=False)
    antsims_cmp[name].spectrum.plot(fig=fig, ax=ax[1,i],ylabel=False)
    
    calobs.plot_calibrated_temp(load, ax=ax[2,i],ylabel=False)
    calobs.plot_calibrated_temp(antsims_cmp[name], ax=ax[2,i],ylabel=False)
    
ax[0,0].set_ylabel("Thermistor Temp (K)")
ax[1,0].set_ylabel("Uncalibrated Temp (K)")
ax[2,0].set_ylabel("Calibrated Temp (K)")
ax[0,1].legend()
plt.tight_layout()

## S11

In [ ]:
fig, ax = plt.subplots(4,5, sharex=True, figsize=(16, 7))
warnings.filterwarnings("ignore", category=DeprecationWarning)

for i, (name, source) in enumerate(calobs._loads.items()):
    source.reflections.plot_residuals(fig=fig, ax=ax[:,i], title=False, decade_ticks=False, ylabels=not i, color_abs='C0', color_diff='C0',label='Primary' if not i else None)
    calobs_cmp._loads[name].reflections.plot_residuals(fig=fig, ax=ax[:,i], title=False, decade_ticks=False,ylabels=not i, color_abs='C1', color_diff='C1', label='Comparison' if not i else None)
    ax[0,i].set_title(name)
    
calobs.lna.plot_residuals(fig=fig, ax=ax[:,-1], title=False, decade_ticks=False,ylabels=False, color_abs='C0', color_diff='C0')
calobs_cmp.lna.plot_residuals(fig=fig, ax=ax[:,-1], title=False, decade_ticks=False,ylabels=False, color_abs='C1', color_diff='C1')

ax[0,-1].set_title("LNA");

In [ ]:
fig, ax = plt.subplots(4,len(antsims), sharex=True, figsize=(16, 7))
warnings.filterwarnings("ignore", category=DeprecationWarning)

for i, (name, source) in enumerate(antsims.items()):
    source.reflections.plot_residuals(fig=fig, ax=ax[:,i], title=False, decade_ticks=False, ylabels=not i, color_abs='C0', color_diff='C0',label='Primary' if not i else None)
    antsims_cmp[name].reflections.plot_residuals(fig=fig, ax=ax[:,i], title=False, decade_ticks=False,ylabels=not i, color_abs='C1', color_diff='C1', label='Comparison' if not i else None)
    ax[0,i].set_title(name)
    

## Calibration

In [ ]:
fig = calobs.plot_coefficients()
ax = fig.axes
calobs_cmp.plot_coefficients(fig=fig, ax=ax);
lines = [Line2D([0], [0], color=f'C{i}', linewidth=2, linestyle='-') for i in range(2)]
labels = ['Primary', 'Comparison']
ax[0].legend(lines, labels);

## List of All Files Used

In [ ]:
import os
def list_files(startpath, filter_list):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(str(startpath), '').count(os.sep)
        indent = ' ' * 4 * (level)
        ff = [f for f in files if any(str(x).startswith(str(Path(root)/f)) for x in filter_list)]
        dd = [d for d in dirs if any(str(x).startswith(str(Path(root)/d)) for x in filter_list)]
        if ff or dd:
            print('{}{}/'.format(indent, os.path.basename(root)))
        if ff:
            subindent = ' ' * 4 * (level + 1)
            for f in sorted(ff):
                print('{}{}'.format(subindent, f))

In [ ]:
fl = calobs.io.list_of_files[0]
print("Primary Observation")
print('===================')
list_files(Path(fl.joinpath(*fl.parts[:3])), calobs.io.list_of_files)

In [ ]:
fl = calobs_cmp.io.list_of_files[0]
print("Comparison Observation")
print('======================')
list_files(Path(fl.joinpath(*fl.parts[:3])), calobs_cmp.io.list_of_files)